<a href="https://colab.research.google.com/github/anniepeacock/DANSAR/blob/devel/burn_severity/GetData_UAVSAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/anniepeacock/DANSAR/blob/devel/utils/Incidence_Angle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Downloading NISAR Simulated Data (from UAVSAR) & Making GIS-Ready Products**

*This notebook runs through downloading and pre-processing UAVSAR Data to be used for fire severity analysis over the Verdugo Mountains in Los Angeles*

- Inputs: GRD, INC, and Annotation files
- Outputs: Masked, cropped, incidence-angle correction, speckle filtered, geotiffs

----------------

Table of Contents:
0. [Running the Notebook](#s1)
1. [Download UAVSAR Data](#s2)
2. [Generate a HDR File](#s3)
3. [Read in data and plot](#s4)
4. [Crop and export to geotiff](#s5)

<a name="s1"></a>
## Running the Notebook
- This Notebook has both "text" and "code" cells. The text cells have text descriptions about running the notebooks and data interpretation.
- Code cells are a light gray and a "play" button appears in the upper left corner when your mouse is hovered over the cell.
- To run the content in the code cells, **select the play button** in the upper left corner of each code cell or **press shift-enter**.

### Python Packages:
This routine uses the following python libraries. Some are already included in the Google Colab environment and others are installed in the cell below before imported. Downloading new python packages to this environment may take a few minutes to complete.

In [ ]:
!pip install rasterio --quiet        # Install python package "rasterio" into Google Colab environment

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 23.8 MB/s eta 0:00:00


In [ ]:
# Scripts and functions needed from GitHub repos
# To build HDR files
!git clone https://bitbucket.org/nathanmthomas/bucket-of-rs-and-gis-scripts/src/master/BuildUAVSARhdr.py

# To appy Enhanced Lee Speckle Filter
!git clone https://github.com/NaiaraSPinto/VegMapper.git

# To generate geotiffs and apply incidence angle masking
!wget https://raw.githubusercontent.com/anniepeacock/DANSAR/devel/utils/functions.py

Cloning into 'BuildUAVSARhdr.py'...
remote: Enumerating objects: 229, done.
remote: Counting objects: 100% (229/229), done.
remote: Compressing objects: 100% (225/225), done.
remote: Total 229 (delta 120), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (229/229), 50.10 KiB | 1.16 MiB/s, done.
Resolving deltas: 100% (120/120), done.
Cloning into 'VegMapper'...
remote: Enumerating objects: 1575, done.
remote: Counting objects: 100% (95/95), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 1575 (delta 34), reused 74 (delta 28), pack-reused 1480
Receiving objects: 100% (1575/1575), 154.23 MiB | 12.98 MiB/s, done.
Resolving deltas: 100% (849/849), done.
Updating files: 100% (64/64), done.
--2024-05-13 21:37:57--  https://raw.githubusercontent.com/anniepeacock/DANSAR/devel/utils/functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (

In [ ]:
import rasterio as rio               # Reading and writing raster formats
import numpy as np                   # Numeric and mathematic functions
import matplotlib.pyplot as plt      # Plotting, making graphs
import getpass                       # Discreet way to enter username and password
import os
from osgeo import gdal
from skimage import filters          # For otsu thresholding
import math
import geopandas as gpd
from matplotlib.patches import Patch          # Creates and plots visualization i.e. legends or figures - (Section 3, 4)
from rasterio.plot import plotting_extent     # Creates an object to plot raster and vector data together - (Section 3. HV Change Detection)
import requests
from osgeo import gdal
from rasterio.plot import show_hist
from rasterio.mask import mask as rio_mask
from shapely.geometry import mapping
from rasterio.plot import show

# Read in functions from functions.py script
from functions import * #download_files, process_grd_files, process_inc_files, mask_and_save, convert_db

# Read in enhanced lee filter from VegMapper
import sys
path_to_module = '/content/VegMapper/vegmapper/core/'
sys.path.append(path_to_module)
from filter import enhanced_lee

<a name="s2"></a>
# 1. Download UAVSAR and NISAR Simulated Data

An Earthdata account is required to download the UAVSAR or NISAR Simulated data stored at ASF to this notebook environment. Create an account here, if needed: https://urs.earthdata.nasa.gov//users/new

In [ ]:
username = getpass.getpass(prompt='Earthdata username:')
password = getpass.getpass(prompt='Earthdata password:')

Earthdata username:··········
Earthdata password:··········


Now, download the UAVSAR data and its corresponding annotation file using wget and your Earthdata username and password. The annotation file (.ann) is required to generate a HDR (.hdr) file. This hdr file stores the georeferencing information of an associated raw raster file and is needed to make the UAVSAR GRD files (.grd) GIS-readable.


Browse for UAVSAR data of interest here: https://uavsar.jpl.nasa.gov/cgi-bin/data.pl

* To download here, replace the links below with the new data of interest

In [3]:
# UAVSAR GRDs and corresponding annotation files (needed to make HDR files and make GRDs GIS-compatible)
uasvar_urls = [
    "https://uavsar.asf.alaska.edu/UA_SanAnd_08525_14158_003_141023_L090_CX_01/SanAnd_08525_14158_003_141023_L090HVHV_CX_01.grd",
    "https://uavsar.asf.alaska.edu/UA_SanAnd_08525_14158_003_141023_L090_CX_01/SanAnd_08525_14158_003_141023_L090_CX_01.ann",
    "https://uavsar.asf.alaska.edu/UA_SanAnd_26526_14092_007_140624_L090_CX_01/SanAnd_26526_14092_007_140624_L090HVHV_CX_01.grd",
    "https://uavsar.asf.alaska.edu/UA_SanAnd_26526_14092_007_140624_L090_CX_01/SanAnd_26526_14092_007_140624_L090_CX_01.ann",

    "https://uavsar.asf.alaska.edu/UA_SanAnd_08525_17122_003_171102_L090_CX_01/SanAnd_08525_17122_003_171102_L090HVHV_CX_01.grd",
    "https://uavsar.asf.alaska.edu/UA_SanAnd_08525_17122_003_171102_L090_CX_01/SanAnd_08525_17122_003_171102_L090_CX_01.ann",
    "https://uavsar.asf.alaska.edu/UA_SanAnd_26526_17122_004_171102_L090_CX_01/SanAnd_26526_17122_004_171102_L090HVHV_CX_01.grd",
    "https://uavsar.asf.alaska.edu/UA_SanAnd_26526_17122_004_171102_L090_CX_01/SanAnd_26526_17122_004_171102_L090_CX_01.ann"
]

destination_directory = "./"

download_files(uavsar_urls, destination_directory, username, password)

# can also download one at a time with wget:
# !wget --user {username} --password {password} https://uavsar.asf.alaska.edu/UA_SanAnd_08525_14158_003_141023_L090_CX_01/SanAnd_08525_14158_003_141023_L090HVHV_CX_01.grd

In [ ]:
# Download incidence angle files
# NISAR Simulated products do not have their own generated indicence angle files - will need to find the *.inc files from the corresponding UAVSAR data lines

# Incidence Angle Files for each line
inc_urls = [
    "https://uavsar.asf.alaska.edu/UA_SanAnd_26526_17122_004_171102_L090_CX_01/SanAnd_26526_17122_004_171102_L090_CX_01.inc",
    "https://uavsar.asf.alaska.edu/UA_SanAnd_08525_23008_011_230712_L090_CX_01/SanAnd_08525_17122_003_171102_L090_CX_01.inc"
]

download_files(inc_urls, destination_directory, username, password)

Failed to download: SanAnd_26526_17122_004_171102_L090_CX_01.inc, Status code: 401
Failed to download: SanAnd_08525_17122_003_171102_L090_CX_01.inc, Status code: 401


<a name="s3"></a>
# 2. Generate a HDR file

Generate HDR files for each of the GRDs, so they can be GIS-compatible and converted to geotiffs.

In [ ]:
!python3 "/content/BuildUAVSARhdr.py/BuildUAVSARhdr.py" -h

UAVSAR.py is written by Nathan Thomas (nmthomas28@gmail.com, @DrNASApants)  formerly of the Aberystwyth University Earth Observation and Ecosystems Dynamics Laboratory (@AU_EarthObs) as part of a visiting research program at NASA JPL. Ammended to process UAVSAR InSAR pair data by Yang Zheng (April 2015).
Use '-h' for help and required input parameters

usage: BuildUAVSARhdr.py [-h] [-i INPUT] [-r UAVSAR] [-p POLARIZATION]

options:
  -h, --help            show this help message and exit
  -i INPUT, --input INPUT
                        Specify the input UAVSAR ann file
  -r UAVSAR, --uavsar UAVSAR
                        Specify the input UAVSAR radar file
  -p POLARIZATION, --polarization POLARIZATION
                        Specify the input UAVSAR polarization in UPPERCASE (i.e HHHV, HHHH, VVVV,
                        AMP1, AMP2, COR, UNW, HGT, INT)


In [ ]:
def run_hdr_script(directory, polarization):
    file_data = []

    # Find GRD and annotation files
    for filename in os.listdir(directory):
        if filename.endswith('.grd'):
            grd_file = filename
            annotation_file = filename.replace(f'_L090{polarization}_', '_L090_').replace('.grd', '.ann')
            file_data.append((grd_file, annotation_file))
        elif filename.endswith('.inc'):
            grd_file = filename
            annotation_file = filename.replace(f'_L090{polarization}_', '_L090_').replace('.inc', '.ann')
            file_data.append((grd_file, annotation_file))

    # Run HDR script for each file
    for grd_file, annotation_file in file_data:
        !python3 "/content/BuildUAVSARhdr.py/BuildUAVSARhdr.py" -i {annotation_file} -r {grd_file} -p {polarization}

# Run HDR script for HVHV polarization
run_hdr_script('./', 'HVHV')

# Run HDR script for HHHH polarization
run_hdr_script('./', 'HHHH')

# Run HDR script for incidence angle files (assuming .inc files)
run_hdr_script('./', 'HVHV')  # You can put whichever polarization, using HVHV as a placeholder here

UAVSAR.py is written by Nathan Thomas (nmthomas28@gmail.com, @DrNASApants)  formerly of the Aberystwyth University Earth Observation and Ecosystems Dynamics Laboratory (@AU_EarthObs) as part of a visiting research program at NASA JPL. Ammended to process UAVSAR InSAR pair data by Yang Zheng (April 2015).
Use '-h' for help and required input parameters

UPPER LEFT LAT =  34.4433
UPPER LEFT LONG =  -119.2884
SAMPLES = 4136
Lines = 32329
PIXEL SIZE =  0.0001
DATATYPE =  4
SanAnd_08525_14158_003_141023_L090HVHV_CG_138A_02.grd.hdr
Writing output HDR file...
Output HDR file = SanAnd_08525_14158_003_141023_L090HVHV_CG_138A_02.grd.hdr

Thank you for using UAVSAR.py

UAVSAR.py is written by Nathan Thomas (nmthomas28@gmail.com, @DrNASApants)  formerly of the Aberystwyth University Earth Observation and Ecosystems Dynamics Laboratory (@AU_EarthObs) as part of a visiting research program at NASA JPL. Ammended to process UAVSAR InSAR pair data by Yang Zheng (April 2015).
Use '-h' for help and requi

<a name="s5"></a>
# 4. Crop the data and export subset to geotiff

In [ ]:
# Define input directory and output directory
input_directory = './' # Directory containing GRD and INC files
output_directory = './'  # Directory to save processed files

# Define extent and projection details
upper_left_x = 373624.5024220281047747
upper_left_y = 3780585.1359808142296970
lower_right_x = 387664.5024220281047747
lower_right_y = 3791835.1359808142296970
window = (upper_left_x, upper_left_y, lower_right_x, lower_right_y)

projection = 'EPSG:26911' # reprojecting here to be in meters to match the Landsat data that will be reprojected in meters.

In [ ]:
# Process GRD files
process_grd_files(input_directory, output_directory, window, projection)

In [ ]:
# Process incidence angle files
process_inc_files(input_directory, output_directory)

# 5. Incidence Angle Corrections/Masking

Converting sigma0 to gamma0 should remove most of the range/incidence angle dependency.

gamma0 = sigma0 / cos(incidence angle)

In [ ]:
mask_and_save(input_directory, output_directory)

In [ ]:
grd = rio.open('/content/SanAnd_26526_17122_004_171102_L090HVHV_CG_138A_02_masked.tif')
grd = grd.read(1)
plt.imshow(grd, vmin=0, vmax=0.1, cmap="Greys_r")

grd = rio.open('/content/SanAnd_08525_17122_003_171102_L090HVHV_CG_138A_02_masked.tif')
grd = grd.read(1)
plt.imshow(grd, vmin=0, vmax=0.1, cmap="Greys_r")

## 2014
# Open the two input rasters using rasterio
with rio.open('/content/SanAnd_08525_14158_003_141023_L090HVHV_CG_138A_02_masked.tif') as src1:
    arr1 = src1.read(1, masked=True)
    meta1 = src1.meta

with rio.open('/content/SanAnd_26526_14092_007_140624_L090HVHV_CG_138A_02_masked.tif') as src2:
    arr2 = src2.read(1, masked=True)
    meta2 = src2.meta

# Merge the two arrays using numpy's maximum function to take the maximum value at each pixel
merged_arr_2014 = np.nanmax((arr1, arr2), axis=0)

## 2017
# Open the two input rasters using rasterio
with rio.open('/content/SanAnd_26526_17122_004_171102_L090HVHV_CG_138A_02_masked.tif') as src1:
    arr1 = src1.read(1, masked=True)
    meta1 = src1.meta

with rio.open('/content/SanAnd_08525_17122_003_171102_L090HVHV_CG_138A_02_masked.tif') as src2:
    arr2 = src2.read(1, masked=True)
    meta2 = src2.meta

# Merge the two arrays using numpy's maximum function to take the maximum value at each pixel
merged_arr_2017 = np.nanmax((arr1, arr2), axis=0)

In [ ]:
fig, axs = plt.subplots(figsize=(20,10))
plt.imshow(merged_arr_2014, vmin=0, vmax=0.1, cmap="Greys_r")

In [ ]:
fig, axs = plt.subplots(figsize=(20,10))
plt.imshow(merged_arr_2017, vmin=0, vmax=0.1, cmap="Greys_r")

In [ ]:
plt.figure(figsize=(8, 6))
plt.hist(merged_arr_2017.flatten(), range = [-0.25, 1.5], bins=200, alpha=0.7, color='blue')
plt.hist(merged_arr_2014.flatten(), range = [-0.25, 1.5], bins=200, alpha=0.7, color='red')
plt.xlabel('Pixel Values')
plt.ylabel('Frequency')
plt.title('Histogram of dNBR 2014 & 2017')
plt.show()

# Lee Filter

In [ ]:
lee_merged_arr_2014 = enhanced_lee(merged_arr_2014, 5, 1)
lee_merged_arr_2017 = enhanced_lee(merged_arr_2017, 5, 1)

In [ ]:
## HV log
HV_log = np.log(np.divide(lee_merged_arr_2014, lee_merged_arr_2017))

## HV log ratio in dB
HV_log_dB = np.divide(convert_db(lee_merged_arr_2014), convert_db(lee_merged_arr_2017))

In [ ]:
# Open the reference raster
with rasterio.open(reference_raster) as src:
    # Get metadata from the reference raster
    meta = src.meta

    # Update metadata to match the NumPy array
    meta.update({
        'dtype': HV_log_2014_2017.dtype,
        'count': 1  # Assuming there's only one band
    })

    # Write the NumPy array to a new GeoTIFF file
    with rasterio.open("HV_log_uavsar.tif", 'w', **meta) as dst:
        dst.write(HV_log, 1)  # Assuming there's only one band (band index 1)

In [ ]:
HV_log_2014_2017 = np.log(np.divide(lee_merged_arr_2014, lee_merged_arr_2017))

fig, axs = plt.subplots(figsize=(20,10))
fig.suptitle('HV Change Detection')

HV_log_plt = axs.imshow(HV_log_2014_2017, vmin=0, vmax=2, cmap="Spectral_r")
fig.colorbar(HV_log_plt, ax=axs, shrink=0.5)